In this notebook, you can consume and see the output of the messages of the kafka topics produced in the script/run-sentiment.sh. Execute the `./kafka-flink-sentiment.sh` script at the project root first.


In [ ]:
# jupyterlab_vim

In [5]:
from kafka import KafkaConsumer
from kafka.admin import KafkaAdminClient

bootstrap_servers = "kafka-broker:9092"

admin_client = KafkaAdminClient(
    bootstrap_servers=bootstrap_servers,
    client_id="demo",
)

## input into kafka source topic


In [6]:
topic_name = "sentiment"
source_topic = KafkaConsumer(
    topic_name,
    bootstrap_servers=["kafka-broker:9092"],
    auto_offset_reset="earliest",
    enable_auto_commit=False,
)

batch = source_topic.poll(timeout_ms=1000)
for partition, messages in batch.items():
    for message in messages:
        print(message.value.decode("utf-8"))
source_topic.close()

﻿The Project Gutenberg EBook of By Sheer Pluck, by G. A. Henty

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: By Sheer Pluck
A Tale of the Ashanti War

Author: G. A. Henty

Release Date: July, 2005 [EBook #8576]
Posting Date: July 24, 2009
Last Updated: March 11, 2018

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK BY SHEER PLUCK ***




Produced by Martin Robb





BY SHEER PLUCK

A TALE OF THE ASHANTI WAR


By G. A. Henty




CHAPTER I: A FISHING EXCURSION


“Now, Hargate, what a fellow you are! I've been looking for you
everywhere. Don't you know it's the House against the Town boys. It's
lucky that the Town have got the first innings; they began a quarter of
an hour ago.”

“How tiresome!” Frank Hargate said. “I was watching a mos

In [ ]:
## output of sentiment analysis

In [7]:
result_topic = KafkaConsumer(
    "sentiment_analysis",
    bootstrap_servers=["kafka-broker:9092"],
    auto_offset_reset="earliest",
    enable_auto_commit=False,
)

result_batch = result_topic.poll(timeout_ms=1000)
for partition, messages in result_batch.items():
    for message in messages:
        print(message.value.decode("utf-8"))
source_topic.close()

Post: with their heads bent down. More than one choking sob might have been | Sentiment: Neutral
Post: heard, had the wind lulled, as they thought of the dear ones at home. | Sentiment: Neutral
Post: Suddenly there was a flash of light ahead, and the boom of a gun | Sentiment: Neutral
Post: directly afterwards came upon their ears. Then a rocket soared up into | Sentiment: Neutral
Post: the air. | Sentiment: Neutral
Post:  | Sentiment: Neutral
Post: “There is a vessel on the sands,” Frank exclaimed. “Let us make for her. | Sentiment: Neutral
Post: If we can get on board we shall have a better chance than here.” | Sentiment: Neutral
Post:  | Sentiment: Neutral
Post: The boys again bent to their oars, and Frank tried to steer exactly for | Sentiment: Neutral
Post: the spot whence the rocket had gone up. Presently another gun flashed | Sentiment: Neutral
Post: out. | Sentiment: Neutral
Post:  | Sentiment: Neutral
Post: “There she is,” he said. “I can see her now against the line of | Sent